Создание сводных таблиц



In [44]:
import pandas as pd
import os
from docxtpl import DocxTemplate
import csv
from tkinter import *
from tkinter import filedialog
from tkinter import messagebox
from tkinter import ttk
import openpyxl
import time
from openpyxl.chart.label import DataLabelList
from openpyxl.chart import BarChart, Reference, PieChart, PieChart3D, Series
# Показываем все колонки
pd.set_option('display.max_columns', None)

In [45]:
def calculate_age(born):
    """
    Функция для расчета текущего возраста взято с https://stackoverflow.com/questions/2217488/age-from-birthdate-in-python/9754466#9754466
    :param born: дата рождения
    :return: возраст
    """

    try:
        today = date.today()
        return today.year - born.year - ((today.month, today.day) < (born.month, born.day))
    except:
        messagebox.showerror('ЦОПП Бурятия', 'Отсутствует или некорректная дата рождения слушателя\nПроверьте файл!')
        quit()


def convert_date(cell):
    """
    Функция для конвертации даты в формате 1957-05-10 в формат 10.05.1957(строковый)
    """
    string_date = datetime.datetime.strftime(cell, '%d.%m.%Y')
    return string_date

def create_initials(fio):
    """
    Функция для создания инициалов для использования в договорах
    формат фио -Будаев Олег Тимурович выходной формат О.Т. Будаев
    """
    #Создаем 3 переменные

    initials_firstname = ''
    initials_middlename = ''
    initials_lastname = ''

    # Сплитим по пробелу
    lst_fio = fio.split()
    # Если ФИО стандартное
    if len(lst_fio) == 3:

        lastname = lst_fio[0]
        firstname = lst_fio[1]
        middlename = lst_fio[2]
        # Создаем инициалы
        initials_firstname = firstname[0].upper()
        initials_middlename = middlename[0].upper()
        initials_lastname = lastname
        # Возвращаем полученную строку
        return f'{initials_firstname}.{initials_middlename}. {initials_lastname}'
    elif len(lst_fio) == 2:
        lastname = lst_fio[0]
        firstname = lst_fio[1]


        initials_firstname = firstname[0].upper()
        initials_lastname = lastname
        return f'{initials_firstname}. {initials_lastname}'
    elif len(lst_fio) == 1:
        lastname = lst_fio[0]
        initials_lastname = lastname
        return f'{initials_lastname}'
    else:
        messagebox.showerror('ЦОПП Бурятия','Проверьте правильность написания ФИО в столбце ФИО_именительный')
        quit()
def counting_total_student(dpo_df, po_df):
    """
    Функция для подсчета общего количества студентов обучающихся в цопп, и количества обучающихся по ДПО И ПО
    :param dpo_df: датафрейм ДПО
    :param po_df: датафрейм ПО
    :return: кортеж вида: общее количество обучающихся,количество обучающихся ДПО,количество обучающихся ПО
    """
    # количество по типам
    total_dpo = dpo_df.shape[0]
    total_po = po_df.shape[0]
    # общее количество
    total = total_dpo + total_po

    return total, total_dpo, total_po


def counting_type_of_training(dpo, po):
    """
    Функция для создания сводной таблицы по категориям направление подготовки, название программы,количество обучающихся
    :param dpo: датафрейм ДПО
    :param po: датафрейм ПО
    :return: датафрейм сводной таблицы
    """
    # Создаем сводные таблицы
    dpo_svod_category_and_name = pd.pivot_table(dpo, index=[
        'Дополнительная_профессиональная_программа_повышение_квалификации_профессиональная_переподготовка',
        'Наименование_дополнительной_профессиональной_программы'],
                                                values=['ФИО_именительный'],
                                                aggfunc='count')
    po_svod_category_and_name = pd.pivot_table(po,
                                               index=['Программа_профессионального_обучения_направление_подготовки',
                                                      'Наименование_программы_профессионального_обучения'],
                                               values=['ФИО_именительный'],
                                               aggfunc='count')

    # Добавляем цифровой индекс
    dpo_svod_category_and_name = dpo_svod_category_and_name.reset_index()
    po_svod_category_and_name = po_svod_category_and_name.reset_index()
    # Изменяем названия колонок, чтобы без проблем соединить 2 датафрейма
    dpo_svod_category_and_name.columns = ['Направление подготовки', 'Название программы', 'Количество обученных']
    po_svod_category_and_name.columns = ['Направление подготовки', 'Название программы', 'Количество обученных']
    # Создаем единую сводную таблицу
    general_svod_category_and_name = pd.concat([dpo_svod_category_and_name, po_svod_category_and_name],
                                               ignore_index=True)
    return general_svod_category_and_name


def counting_total_sex(dpo, po):
    """
    Функция для подсчета количества мужчин и женщин
    :param dpo: датафрейм ДПО
    :param po: датафрейм ПО
    :return: датафрейм сводной таблицы
    """
    # Создаем сводные таблицы
    dpo_total_sex = pd.pivot_table(dpo, index=['Пол_получателя'],
                                   values=['ФИО_именительный'],
                                   aggfunc='count')
    po_total_sex = pd.pivot_table(po, index=['Пол_получателя'],
                                  values=['ФИО_именительный'],
                                  aggfunc='count')
    # Извлекаем индексы
    dpo_total_sex = dpo_total_sex.reset_index()
    po_total_sex = po_total_sex.reset_index()
    # Переименовываем колонки
    dpo_total_sex.columns = ['Пол', 'Количество']
    po_total_sex.columns = ['Пол', 'Количество']

    # Соединяем в единую таблицу
    general_total_sex = pd.concat([dpo_total_sex, po_total_sex], ignore_index=True)
    # Группируем по полю Пол чтобы суммировать значения
    sum_general_total_sex = general_total_sex.groupby(['Пол']).sum().reset_index()
    return sum_general_total_sex


def counting_age_distribution(dpo, po):
    """
    Функция для подсчета количества обучающихся по возрастным категориям
    :param dpo: датафрейм ДПО
    :param po: датафрейм ПО
    :return: датафрейм сводной таблицы
    """
    # Создаем сводные таблицы
    dpo_age_distribution = pd.pivot_table(dpo, index=['Возрастная_категория'],
                                          values=['ФИО_именительный'],
                                          aggfunc='count')
    po_age_distribution = pd.pivot_table(po, index=['Возрастная_категория'],
                                         values=['ФИО_именительный'],
                                         aggfunc='count')
    # Извлекам индексы
    dpo_age_distribution = dpo_age_distribution.reset_index()
    po_age_distribution = po_age_distribution.reset_index()
    # Меняем колонки
    dpo_age_distribution.columns = ['Возрастная_категория', 'Количество']
    po_age_distribution.columns = ['Возрастная_категория', 'Количество']

    # Создаем единую сводную таблицу
    general_age_distribution = pd.concat([dpo_age_distribution, po_age_distribution], ignore_index=True)
    # Повторно группируем чтобы соединить категории из обеих таблиц
    general_age_distribution = general_age_distribution.groupby(['Возрастная_категория']).sum().reset_index()

    return general_age_distribution


In [46]:
dpo_df = pd.read_excel('Общая таблица слушателей ЦОПП от 08_02_22.xlsx', sheet_name='ДПО')
po_df = pd.read_excel('Общая таблица слушателей ЦОПП от 08_02_22.xlsx',sheet_name='ПО')
dpo_df = dpo_df.fillna('Не заполнено!!!')
po_df = po_df.fillna('Не заполнено!!!')

Мне нужно получить группировку по полям
Дополнительная_профессиональная_программа_повышение_квалификации_профессиональная_переподготовка
Наименование_дополнительной_профессиональной_программы
Количество обучающихся

In [47]:
dpo_svod_category_and_name=pd.pivot_table(dpo_df,index=['Дополнительная_профессиональная_программа_повышение_квалификации_профессиональная_переподготовка',
                             'Наименование_дополнительной_профессиональной_программы'],
              values=['ФИО_именительный'],
              aggfunc='count')

In [48]:
po_svod_category_and_name = pd.pivot_table(po_df,index=['Программа_профессионального_обучения_направление_подготовки','Наименование_программы_профессионального_обучения'],
                                          values=['ФИО_именительный'],
                                          aggfunc='count')

In [49]:
print(dpo_svod_category_and_name)

                                                                                                       ФИО_именительный
Дополнительная_профессиональная_программа_повыш... Наименование_дополнительной_профессиональной_пр...                  
Повышение квалификации                             Профессиональное самоопределение личности на ур...                24
Профессиональная переподготовка                    Агроном                                                            4


In [50]:
print(po_svod_category_and_name)

                                                                                                      ФИО_именительный
Программа_профессионального_обучения_направлени... Наименование_программы_профессионального_обучения                  
Программа переподготовки рабочих, служащих         Сварщик                                                           6


In [51]:
dpo_svod_category_and_name = dpo_svod_category_and_name.reset_index()
po_svod_category_and_name = po_svod_category_and_name.reset_index()

In [52]:
dpo_svod_category_and_name.columns=['Направление подготовки','Название программы','Количество обученных']
po_svod_category_and_name.columns=['Направление подготовки','Название программы','Количество обученных']

In [53]:
general_svod_category_and_name = pd.concat([dpo_svod_category_and_name,po_svod_category_and_name],ignore_index=True)

In [54]:
general_svod_category_and_name

,Направление подготовки,Название программы,Количество обученных
0,Повышение квалификации,Профессиональное самоопределение личности на у...,24
1,Профессиональная переподготовка,Агроном,4
2,"Программа переподготовки рабочих, служащих",Сварщик,6


Создание сводной таблицы по индикаторам


In [55]:
dpo_svod_category_and_name=pd.pivot_table(dpo_df,index=['Дополнительная_профессиональная_программа_повышение_квалификации_профессиональная_переподготовка',
                             'Наименование_дополнительной_профессиональной_программы','Пол_получателя'],
              values=['ФИО_именительный'],
              aggfunc='count')

In [56]:
dpo_svod_category_and_name

ФИО_именительный
Дополнительная_профессиональная_программа_повыш... Наименование_дополнительной_профессиональной_пр... Пол_получателя                  
Повышение квалификации                             Профессиональное самоопределение личности на ур... Жен                           19
                                                                                                      Муж                            5
Профессиональная переподготовка                    Агроном                                            Жен                            4

In [64]:
pd.pivot_table(dpo_df,index=['Пол_получателя'],
              values=['ФИО_именительный'],
              aggfunc='count')

,Укрупненные_группы_специальностей,ФИО_именительный
Пол_получателя,,
Жен,23,23
Муж,5,5


In [76]:
pd.pivot_table(dpo_df,index=['Пол_получателя','Является_ли_слушатель_руководителем',
                             'Наименование_дополнительной_профессиональной_программы',
                             'Уровень_образования_ВО_СПО'],
              values=['ФИО_именительный','Текущий_возраст'],
              aggfunc={'ФИО_именительный':'count','Текущий_возраст':'mean'})

In [77]:
pd.pivot_table(dpo_df,index=['Категория_слушателя','Является_ли_слушатель_руководителем'],
              values=['ФИО_именительный'],
              aggfunc={'ФИО_именительный':'count'})

ФИО_именительный
Категория_слушателя                          Является_ли_слушатель_руководителем                  
ГГС или замещающий государственную должность Не руководитель                                     5
                                             Руководитель                                        1
Педагогический работник                      Не руководитель                                    13
незанятое лицо по напр СЗ                    руководитель                                        4
студент СПО                                  Не руководитель                                     3
                                             Руководитель                                        2

In [58]:
total_students, total_students_dpo, total_students_po = counting_total_student(dpo_df, po_df)

# Количество обучившихся ДПО и ПО

# Получение количества обучившихся по видам
df_counting_type_and_name_trainning = counting_type_of_training(dpo_df, po_df)

# Создаем новый excel файл
wb = openpyxl.Workbook()

# Получаем активный лист
sheet = wb.active
sheet.title = 'Сводные данные'

# Начинаем заполнение листа
# Заполняем количество обучившихся, общее и по типам
sheet['A1'] = 'Наименование показателя'
sheet['A2'] = 'Количество прошедших обучение ДПО'
sheet['A3'] = 'Количество прошедших обучение ПО'
sheet['A4'] = 'Общее количество прошедших обучение в ЦОПП'

sheet['B1'] = 'Количество обучающихся'
sheet['B2'] = total_students_dpo
sheet['B3'] = total_students_po
sheet['B4'] = total_students

# Добавляем круговую диаграмму
pie_main = PieChart()
labels = Reference(sheet, min_col=1, min_row=2, max_row=3)
data = Reference(sheet, min_col=2, min_row=2, max_row=3)

# Для отображения данных на диаграмме
series = Series(data, title='Series 1')
pie_main.append(series)

s1 = pie_main.series[0]
s1.dLbls = DataLabelList()
s1.dLbls.showVal = True

pie_main.add_data(data, titles_from_data=True)
pie_main.set_categories(labels)
pie_main.title = 'Распределение обучившихся'
sheet.add_chart(pie_main, 'F2')
# # Добавляем таблицу с по направлениям

sheet['A7'] = 'Вид обучения'
sheet['B7'] = 'Название программы'
sheet['C7'] = 'Количество обучающихся'

for row in df_counting_type_and_name_trainning.values.tolist():
    sheet.append(row)
# Получаем последние активные ячейки чтобы записывалось по порядку и не налазило друг на друга
min_column = wb.active.min_column
max_column = wb.active.max_column
min_row = wb.active.min_row
max_row = wb.active.max_row

sheet[f'A{max_row + 2}'] = 'Общее распределение прошедших обучение по полу'
total_sex = counting_total_sex(dpo_df, po_df)
# Добавляем в файл таблицу с распределением по полам
for row in total_sex.values.tolist():
    sheet.append(row)

# Получаем последние активные ячейки чтобы записывалось по порядку и не налазило друг на друга
min_column = wb.active.min_column
max_column = wb.active.max_column
min_row = wb.active.min_row
max_row = wb.active.max_row

# Добавляем таблицу с разбиением по возрастам
sheet[f'A{max_row + 2}'] = 'Общее распределение обучающихся по возрасту'
age_distribution = counting_age_distribution(dpo_df, po_df)
for row in age_distribution.values.tolist():
    sheet.append(row)

# Добавляем круговую диаграмму
pie_age = PieChart()
# Для того чтобы не зависело от количества строк в предыдущих таблицах
labels = Reference(sheet, min_col=1, min_row=max_row + 3, max_row=max_row + 2 + len(age_distribution))
data = Reference(sheet, min_col=2, min_row=max_row + 3, max_row=max_row + 2 + len(age_distribution))
# Для отображения данных на диаграмме
series = Series(data, title='Series 1')
pie_age.append(series)

s1 = pie_age.series[0]
s1.dLbls = DataLabelList()
s1.dLbls.showVal = True

pie_age.add_data(data, titles_from_data=True)
pie_age.set_categories(labels)
pie_age.title = 'Распределение обучившихся по возрастным категориям'

sheet.add_chart(pie_age, f'F{max_row + 2}')

min_column = wb.active.min_column
max_column = wb.active.max_column
min_row = wb.active.min_row
max_row = wb.active.max_row



sheet.column_dimensions['A'].width = 50
sheet.column_dimensions['B'].width = 30

In [59]:
# Сохраняем файл
t = time.localtime()
current_time = time.strftime('%H_%M_%S', t)
wb.save(f'Сводный отчет {current_time}.xlsx')
